# Concatenating multiple feature extraction methods

Adapted from http://scikit-learn.org/stable/auto_examples/feature_stacker.html

In many real-world examples, there are many ways to extract features from a dataset. Often it is beneficial to combine several methods to obtain good performance. This example shows how to use FeatureUnion to combine features obtained by PCA and univariate selection.

Combining features using this transformer has the benefit that it allows cross validation and grid searches over the whole process.

The combination used in this example is not particularly helpful on this dataset and is only used to illustrate the usage of FeatureUnion.

In [1]:
using ScikitLearn
using ScikitLearn.GridSearch: GridSearchCV
using ScikitLearn.Pipelines: Pipeline, FeatureUnion

@sk_import svm: SVC
@sk_import datasets: load_iris
@sk_import decomposition: PCA
@sk_import feature_selection: SelectKBest

In [2]:
# Original Python Author: Andreas Mueller <amueller@ais.uni-bonn.de>
#
# License: BSD 3 clause

iris = load_iris()

X, y = iris["data"], iris["target"]

# This dataset is way to high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=1)

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = transform(fit!(combined_features, X, y), X)

svm = SVC(kernel="linear")

# Do grid search over k, n_components and C:

pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = Dict(:features__pca__n_components=>[1, 2, 3],
                  :features__univ_select__k=>[1, 2],
                  :svm__C=>[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid; verbose=10, refit=true)

fit!(grid_search, X, y)

print(grid_search.best_estimator_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1, score=0.96078  -  0.1s
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1, score=0.90196  -  0.0s
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1, score=0.97917  -  0.0s
[CV] features__pca__n_components=1, svm__C=1.0, features__univ_select__k=1
[CV] features__pca__n_components=1, svm__C=1.0, features__univ_select__k=1, score=0.94118  -  0.0s
[CV] features__pca__n_components=1, svm__C=1.0, features__univ_select__k=1
[CV] features__pca__n_components=1, svm__C=1.0, features__univ_select__k=1, score=0.92157  -  0.0s
[CV] features__pca__n_components=1, svm__C=1.0, features__univ_select